# Combine national and NERC data
Take the various file (EIA facilities, EIA state-level totals, EPA emissions, etc) and combine them to determine total generation, CO₂ emissions, etc nationally and at NERC regions.

In [7]:
import pandas as pd
import numpy as np
import os
from os.path import join
import sys
import json

idx = pd.IndexSlice
cwd = os.getcwd()
data_path = join(cwd, '..', 'Data storage')

### Date string for filenames
This will be inserted into all filenames (reading and writing)

In [8]:
file_date = '2018-03-06'

In [9]:
%load_ext watermark
%watermark -iv -v

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
pandas      0.22.0
numpy       1.14.2
json        2.0.9
CPython 3.6.4
IPython 6.2.1


In [10]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# add the 'src' directory as one where we can import modules
src_dir = join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [63]:
%aimport Data.make_data
from Data.make_data import states_in_nerc
%aimport Analysis.index
from Analysis.index import facility_emission_gen, group_facility_data, add_quarter
%aimport Analysis.index
from Analysis.index import g2lb, change_since_2005, generation_index
%aimport Analysis.index
from Analysis.index import facility_co2, adjust_epa_emissions, group_fuel_cats
%aimport util.utils
from util.utils import rename_cols, add_facility_location

## Load data

In [13]:
cwd = os.getcwd()
path = join(data_path, 'Derived data',
            'Facility gen fuels and CO2 {}.csv'.format(file_date))
eia_fac = pd.read_csv(path)

In [14]:
rename_cols(eia_fac)

In [15]:
eia_fac.head()

,f,fuel,month,plant id,total fuel (mmbtu),year,generation (mwh),elec fuel (mmbtu),geography,last_updated,lat,lon,prime mover,datetime,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,M,DFO,12,1001,1116.0,2017,114.587,1116.0,USA-IN,2018-02-28T02:03:13-05:00,39.9242,-87.4244,ALL,2017-12-01,4,81646.56,81646.56,81646.56,81646.56
1,M,DFO,11,1001,2772.0,2017,265.428,2772.0,USA-IN,2018-02-28T02:03:13-05:00,39.9242,-87.4244,ALL,2017-11-01,4,202799.52,202799.52,202799.52,202799.52
2,M,DFO,10,1001,4819.0,2017,460.695,4819.0,USA-IN,2018-02-28T02:03:13-05:00,39.9242,-87.4244,ALL,2017-10-01,4,352558.04,352558.04,352558.04,352558.04
3,M,DFO,9,1001,2720.0,2017,300.705,2720.0,USA-IN,2018-02-28T02:03:13-05:00,39.9242,-87.4244,ALL,2017-09-01,3,198995.20,198995.20,198995.20,198995.20
4,M,DFO,8,1001,6273.0,2017,609.358,6273.0,USA-IN,2018-02-28T02:03:13-05:00,39.9242,-87.4244,ALL,2017-08-01,3,458932.68,458932.68,458932.68,458932.68


In [16]:
path = join(data_path, 'Derived data',
            'Monthly EPA emissions {}.csv'.format(file_date))
epa = pd.read_csv(path)

In [17]:
epa.tail()

,CO2_MASS (kg),GLOAD (MW),HEAT_INPUT (mmBtu),MONTH,OP_TIME,ORISPL_CODE,SLOAD (1000lb/hr),YEAR
279427,0.0,0.0,337871.418,8,1494.69,880107,200653.0,2017
279428,0.0,0.0,308866.407,9,1447.30,880107,207589.0,2017
279429,0.0,0.0,327482.620,10,1488.72,880107,216266.0,2017
279430,0.0,0.0,341459.300,11,1440.00,880107,231679.0,2017
279431,0.0,0.0,360386.700,12,1488.00,880107,243984.0,2017


## Adjusted facility emissions and generation
I have both the state and custom fuel categories here, but am only using the state fuel categories for now.

In [18]:
fuel_cat_folder = join(data_path, 'Fuel categories')
state_cats_path = join(fuel_cat_folder, 'State_facility.json')

with open(state_cats_path, 'r') as f:
    state_fuel_cat = json.load(f)
    
custom_cats_path = join(fuel_cat_folder, 'Custom_results.json')
with open(custom_cats_path, 'r') as f:
    custom_fuel_cat = json.load(f)

In [19]:
co2, gen_fuels_custom = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=False)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories
Gen/fuels to custom categories


In [20]:
co2, gen_fuels_state = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=True)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories


In [21]:
co2.tail()

,year,month,plant id,final co2 (kg)
1116371,2017,12,61357,0.0
1116372,2017,12,61407,0.0
1116373,2017,12,61422,0.0
1116374,2017,12,61512,0.0
1116375,2017,12,61561,0.0


In [22]:
gen_fuels_state.tail()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
1564478,WWW,2017,12,57919,296725.0,8933.912,44934.0,34.385278,-80.067778,4,0.0,0.0,27832805.0,4214809.2
1564479,WWW,2017,12,58523,51102.0,1828.000,20790.0,48.387700,-114.240900,4,0.0,0.0,4793367.6,1950102.0
1564480,WWW,2017,12,58574,123607.0,7832.000,123607.0,39.648056,-106.943056,4,0.0,0.0,11594336.6,11594336.6
1564481,WWW,2017,12,58944,0.0,0.000,0.0,30.748333,-94.437500,4,0.0,0.0,0.0,0.0
1564482,WWW,2017,12,60340,500075.0,30748.661,349715.0,31.555900,-84.110300,4,0.0,0.0,46907035.0,32803267.0


### Check generation and fuel consumption totals

Interesting - there is some small part of generation that I'm losing along the way. It's 5 orders of magnitude smaller than the total though.

In [23]:
eia_fac['generation (mwh)'].sum()

67601240020.031395

In [24]:
gen_fuels_state['generation (mwh)'].sum()

67601426136.03142

Fuel consumption is pretty identical though

In [25]:
eia_fac['total fuel (mmbtu)'].sum(), eia_fac['elec fuel (mmbtu)'].sum()

(707222059073.79, 664121418912.0795)

In [26]:
gen_fuels_state['total fuel (mmbtu)'].sum(), gen_fuels_state['elec fuel (mmbtu)'].sum()

(707222059073.7903, 664121418912.08)

## Extra gen/fuels from non-reporting

In [27]:
%aimport Analysis.index
from Analysis.index import extra_emissions_gen

Total EIA generation/fuel consumption and emission factors

In [28]:
cwd = os.getcwd()
path = join(data_path, 'Derived data',
            'EIA country-wide gen fuel CO2 {}.csv'.format(file_date))
eia_total = pd.read_csv(path)

path = join(data_path,
            'Final emission factors.csv')
ef = pd.read_csv(path, index_col=0)

In [29]:
eia_total.head()

,type,year,month,geography,end,f,last_updated,sector,series_id,start,units,generation (MWh),total fuel (mmbtu),elec fuel (mmbtu),all fuel CO2 (kg),elec fuel CO2 (kg),datetime,quarter
0,AOR,2001,1,USA-AK,201712.0,M,2018-02-28T02:03:13-05:00,99.0,ELEC.GEN.AOR-AK-99.M,200101.0,megawatthours,87.00,NaN,NaN,NaN,NaN,2001-01-01,1
1,AOR,2001,1,USA-AL,201712.0,M,2018-02-28T02:03:13-05:00,99.0,ELEC.GEN.AOR-AL-99.M,200101.0,megawatthours,401167.59,NaN,NaN,NaN,NaN,2001-01-01,1
2,AOR,2001,1,USA-AR,201712.0,M,2018-02-28T02:03:13-05:00,99.0,ELEC.GEN.AOR-AR-99.M,200101.0,megawatthours,136530.37,NaN,NaN,NaN,NaN,2001-01-01,1
3,AOR,2001,1,USA-AZ,201712.0,M,2018-02-28T02:03:13-05:00,99.0,ELEC.GEN.AOR-AZ-99.M,200101.0,megawatthours,453.00,NaN,NaN,NaN,NaN,2001-01-01,1
4,AOR,2001,1,USA-CA,201712.0,M,2018-02-28T02:03:13-05:00,99.0,ELEC.GEN.AOR-CA-99.M,200101.0,megawatthours,1717398.41,NaN,NaN,NaN,NaN,2001-01-01,1


### Calculate CO₂, generation, and fuel consumption that is not captured by facilities

In [30]:
extra_co2, extra_gen_fuel = extra_emissions_gen(gen_fuels_state, eia_total, ef)

Results match what I have previously found in the notebooks up on GitHub (Emissions Index repo)

In [31]:
extra_gen_fuel.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
COW  2001 1              16916390.0       1011665.061         13155576.0
          2              14970346.0        911345.865         11576491.0
          3              18320543.0       1153327.913         14764162.0
          4              13530117.0        776339.623         10374008.0
          5              14016024.0        801267.923         10922463.0

In [26]:
extra_gen_fuel.loc[idx['WND',:,:]]

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
WND  2001 1           -3.446894e+06      5.566881e+04      -3.446894e+06
          2           -3.288177e+06      1.130211e+05      -3.288177e+06
          3           -4.798975e+06      6.768812e+04      -4.798975e+06
          4           -6.166244e+06      8.794347e+04      -6.166244e+06
          5           -5.694490e+06      8.393153e+04      -5.694490e+06
          6           -6.120034e+06      7.735354e+04      -6.120034e+06
          7           -5.827621e+06      7.101049e+04      -5.827621e+06
          8           -5.409429e+06      5.369600e+04      -5.409429e+06
          9           -4.550627e+06      4.983153e+04      -4.550627e+06
          10          -5.668965e+06      5.820182e+04      -5.668965e+06
          11          -4.551249e+06      2.999030e+04      -4.551249e+06
          12          -6.077985e+06      2.744590e+04      -6.077985e+06
     2002 1           -8.237968e+06      1.613509e+03      -8.237968e+06
          2           -7.247621e+06      1.495079e+03      -7.247621e+06
          3           -8.650071e+06      1.805048e+03      -8.650071e+06
          4           -1.040601e+07      1.512223e+03      -1.040601e+07
          5           -1.094733e+07      1.523876e+03      -1.094733e+07
          6           -1.143913e+07      1.855552e+03      -1.143913e+07
          7           -9.036664e+06      2.155177e+03      -9.036664e+06
          8           -9.914911e+06      2.062015e+03      -9.914911e+06
          9           -7.466379e+06      1.888244e+03      -7.466379e+06
          10          -7.453861e+06      1.644759e+03      -7.453861e+06
          11          -6.657461e+06      1.522470e+03      -6.657461e+06
          12          -7.665064e+06      1.723605e+03      -7.665064e+06
     2003 1           -6.475789e+06      1.438000e+00      -6.475789e+06
          2           -7.633388e+06     -1.249000e+00      -7.633388e+06
          3           -1.061074e+07     -2.510000e-01      -1.061074e+07
          4           -1.119102e+07      1.789000e+00      -1.119102e+07
          5           -1.030637e+07     -2.610000e-01      -1.030637e+07
          6           -1.072381e+07      6.340000e-01      -1.072381e+07
...                             ...               ...                ...
     2015 7           -1.273987e+08      4.591537e+03      -1.273987e+08
          8           -1.218547e+08      4.083343e+03      -1.218547e+08
          9           -1.299189e+08      3.027036e+04      -1.299189e+08
          10          -1.526123e+08      3.567217e+03      -1.526123e+08
          11          -1.833724e+08      4.449325e+03      -1.833724e+08
          12          -1.872415e+08      5.930401e+03      -1.872415e+08
     2016 1           -1.704424e+08      4.272881e+03      -1.704424e+08
          2           -1.858746e+08      4.495243e+03      -1.858746e+08
          3           -2.024797e+08      7.029687e+03      -2.024797e+08
          4           -1.919616e+08      6.233085e+03      -1.919616e+08
          5           -1.739356e+08      7.389017e+03      -1.739356e+08
          6           -1.504557e+08      6.247802e+03      -1.504557e+08
          7           -1.625816e+08      7.695867e+03      -1.625816e+08
          8           -1.254098e+08      5.074127e+03      -1.254098e+08
          9           -1.514009e+08      4.058139e+03      -1.514009e+08
          10          -1.876898e+08      5.037799e+03      -1.876898e+08
          11          -1.791183e+08      3.746759e+03      -1.791183e+08
          12          -2.136343e+08      4.894474e+03      -2.136343e+08
     2017 1           -7.033578e+07      1.305014e+07      -7.033578e+07
          2           -7.382165e+07      1.417128e+07      -7.382165e+07
          3           -8.841420e+07      1.655041e+07      -8.841420e+07
          4           -8.743995e+07      1.627812e+07      -8.743995e+07
  

## Total CO₂ (national)
Combine adjusted CO₂ at facilities and CO₂ from fuel consumption that is not captured by facility data.

In [32]:
facility_co2 = co2.groupby(['year', 'month']).sum()

In [33]:
facility_co2.tail()

plant id  final co2 (kg)
year month                          
2017 8      59956777    1.685669e+11
     9      60081740    1.386751e+11
     10     60498933    1.275088e+11
     11     61215677    1.233808e+11
     12     62726265    1.438485e+11

In [34]:
extra_co2.loc[idx['NG', :, :],:].tail(n=11)

all fuel co2 (kg)  elec fuel co2 (kg)
type year month                                       
NG   2017 2           6.794671e+09        5.074766e+09
          3           8.979710e+09        7.158304e+09
          4           8.153363e+09        6.487786e+09
          5           9.068398e+09        7.404283e+09
          6           1.091165e+10        9.142059e+09
          7           1.442280e+10        1.249217e+10
          8           1.333576e+10        1.146265e+10
          9           1.133281e+10        9.560062e+09
          10          9.850189e+09        8.133934e+09
          11          9.016542e+09        7.232485e+09
          12          1.006194e+10        8.018525e+09

In [35]:
national_co2 = (facility_co2.loc[:, 'final co2 (kg)']
                + extra_co2.loc[:, 'elec fuel co2 (kg)']
                           .groupby(['year', 'month']).sum())
national_co2.name = 'final co2 (kg)'

In [36]:
national_co2.head()

year  month
2001  1        2.145929e+11
      2        1.799442e+11
      3        1.887242e+11
      4        1.744977e+11
      5        1.888085e+11
Name: final co2 (kg), dtype: float64

## National Index and gen by fuels

In [47]:
extra_gen_fuel.tail()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
WWW  2017 8             -55058738.0       2117348.405        -13990971.0
          9             -49957910.0       1781924.065        -12090584.0
          10            -53801581.0       1900680.564        -13339160.0
          11            -55584275.0       1838602.253        -13369010.0
          12            -59505061.0       2001088.969        -14322102.0

In [46]:
gen_fuels_state.groupby(['year', 'month']).sum().tail()

plant id  total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)  \
year month                                                                      
2017 8      92101054        3.243524e+09      3.303634e+08       3.099425e+09   
     9      92336205        2.821822e+09      2.849385e+08       2.685294e+09   
     10     92793908        2.675856e+09      2.683948e+08       2.533375e+09   
     11     93515311        2.602604e+09      2.601150e+08       2.458400e+09   
     12     95030773        2.969232e+09      2.981346e+08       2.813103e+09   

                      lat            lon  quarter  all fuel fossil co2 (kg)  \
year month                                                                    
2017 8      123777.238892 -303729.803934     9816              1.731163e+11   
     9      123966.011103 -304186.929849     9825              1.436289e+11   
     10     124485.258253 -305617.871689    13160              1.329022e+11   
     11     124758.874949 -306277.678952    13188              1.287741e+11   
     12     125632.300180 -308773.344081    13288              1.498461e+11   

            elec fuel fossil co2 (kg)  all fuel total co2 (kg)  \
year month                                                       
2017 8                   1.664585e+11             1.798772e+11   
     9                   1.372683e+11             1.497610e+11   
     10                  1.263122e+11             1.394534e+11   
     11                  1.222004e+11             1.355091e+11   
     12                  1.427166e+11             1.570085e+11   

            elec fuel total co2 (kg)  
year month                            
2017 8                  1.691147e+11  
     9                  1.396417e+11  
     10                 1.288218e+11  
     11                 1.247023e+11  
     12                 1.453625e+11

### Total national generation (all fuels)
Add generation/fuel consumption reported by facilities (with state-level fuel codes) and extra generation/fuel consumption (which also uses state-level fuel codes).

In [53]:
extra_gen_fuel.merge(extra_co2, left_index=True, right_index=True).columns

Index(['total fuel (mmbtu)', 'generation (mwh)', 'elec fuel (mmbtu)',
       'all fuel co2 (kg)', 'elec fuel co2 (kg)'],
      dtype='object')

In [52]:
gen_fuels_state.groupby(['type', 'year', 'month']).sum().head()

plant id  total fuel (mmbtu)  generation (mwh)  \
type year month                                                   
COW  2001 1       8550284        1.848875e+09      1.762754e+08   
          2       8550284        1.568672e+09      1.488241e+08   
          3       8550284        1.626246e+09      1.541157e+08   
          4       8550284        1.471568e+09      1.398943e+08   
          5       8498259        1.595267e+09      1.507916e+08   

                 elec fuel (mmbtu)           lat         lon  quarter  \
type year month                                                         
COW  2001 1           1.810304e+09  25142.050268 -53175.8593      645   
          2           1.536017e+09  25142.050268 -53175.8593      645   
          3           1.591151e+09  25142.050268 -53175.8593      645   
          4           1.439361e+09  25142.050268 -53175.8593     1290   
          5           1.563698e+09  25147.539168 -53176.2288     1290   

                 all fuel fossil co2 (kg)  elec fuel fossil co2 (kg)  \
type year month                                                        
COW  2001 1                  1.755757e+11               1.719629e+11   
          2                  1.490065e+11               1.459476e+11   
          3                  1.544286e+11               1.511424e+11   
          4                  1.397181e+11               1.367018e+11   
          5                  1.515424e+11               1.485856e+11   

                 all fuel total co2 (kg)  elec fuel total co2 (kg)  
type year month                                                     
COW  2001 1                 1.755757e+11              1.719629e+11  
          2                 1.490065e+11              1.459476e+11  
          3                 1.544286e+11              1.511424e+11  
          4                 1.397181e+11              1.367018e+11  
          5                 1.515424e+11              1.485856e+11

In [66]:
national_gen = (gen_fuels_state
                .groupby(['type', 'year', 'month'])['generation (mwh)'].sum()
                .add(extra_gen_fuel['generation (mwh)'], fill_value=0))

In [67]:
national_gen.head()

type  year  month
COW   2001  1        177287111.0
            2        149735483.0
            3        155269010.0
            4        140670652.0
            5        151592915.0
Name: generation (mwh), dtype: float64

In [68]:
national_gen.groupby(['year', 'month']).sum().tail()

year  month
2017  8        3.842233e+08
      9        3.360709e+08
      10       3.202084e+08
      11       3.080090e+08
      12       3.471760e+08
Name: generation (mwh), dtype: float64

#### Regroup generation from state codes to my custom fuel codes

In [69]:
national_gen = group_fuel_cats(national_gen.reset_index(), custom_fuel_cat,
                               'type', 'fuel category').set_index(['fuel category', 'year', 'month'])

In [70]:
national_gen.tail()

generation (mwh)
fuel category year month                  
Wind          2017 8           13064719.50
                   9           17264878.55
                   10          24814868.01
                   11          23315592.47
                   12          22757913.41

In [61]:
total_gen = national_gen.groupby(['year', 'month']).sum()
total_gen.tail()

generation (mwh)  elec fuel co2 (kg)
year month                                      
2017 8          3.842233e+08        1.794305e+11
     9          3.360709e+08        1.490891e+11
     10         3.202084e+08        1.355305e+11
     11         3.080090e+08        1.304575e+11
     12         3.471760e+08        1.519205e+11

### National Index

In [42]:
national_co2.head()

year  month
2001  1        2.145929e+11
      2        1.799442e+11
      3        1.887242e+11
      4        1.744977e+11
      5        1.888085e+11
Name: final co2 (kg), dtype: float64

In [43]:
total_gen.tail()

generation (mwh)
year month                  
2017 8          3.842233e+08
     9          3.360709e+08
     10         3.202084e+08
     11         3.080090e+08
     12         3.471760e+08

In [71]:
national_index = total_gen.copy()
national_index['final co2 (kg)'] = national_co2
national_index['index (g/kwh)'] = (national_index['final co2 (kg)']
                                   / national_index['generation (mwh)'])
national_index.reset_index(inplace=True)
add_quarter(national_index)
g2lb(national_index)
change_since_2005(national_index)

In [72]:
national_index.tail()

,year,month,generation (mwh),elec fuel co2 (kg),final co2 (kg),index (g/kwh),datetime,quarter,index (lb/mwh),change since 2005
199,2017,8,3.842233e+08,1.794305e+11,1.815390e+11,472.482908,2017-08-01,3,1041.635819,-0.211385
200,2017,9,3.360709e+08,1.490891e+11,1.504959e+11,447.809966,2017-09-01,3,987.241851,-0.252567
201,2017,10,3.202084e+08,1.355305e+11,1.367271e+11,426.994115,2017-10-01,4,941.351226,-0.287310
202,2017,11,3.080090e+08,1.304575e+11,1.316380e+11,427.383523,2017-11-01,4,942.209714,-0.286660
203,2017,12,3.471760e+08,1.519205e+11,1.530523e+11,440.849405,2017-12-01,4,971.896597,-0.264185


In [73]:
path = join(data_path, 'National data',
            'National index {}.csv'.format(file_date))
national_index.to_csv(path, index=False)

### Percent generation by custom fuel type

In [74]:
df_list = []
for fuel in national_gen.index.get_level_values('fuel category').unique():
    percent_gen = national_gen.loc[fuel].divide(total_gen, fill_value=0)
    percent_gen['fuel category'] = fuel
    percent_gen.set_index('fuel category', inplace=True, append=True)
    df_list.append(percent_gen)
percent_gen = pd.concat(df_list)

In [75]:
path = join(data_path, 'National data',
            'National generation {}.csv'.format(file_date))
national_gen.to_csv(path)

path = join(data_path, 'National data',
            'National percent gen {}.csv'.format(file_date))
percent_gen.to_csv(path)

## Facility state and lat/lon file generation
~~Create a .csv with lat/lon and state code for each facility. Need to manually add the NERC region label.~~ ~~I've done this with a spatial join between the lat/lon and NERC shapefiles in QGIS. It should be possible to do the spatial join in [GeoPandas](http://geopandas.org/).  Could also add custom region labels (e.g. eGRID subregions, ISO/RTO boundaries, etc).~~

This isn't possible with a spatial join because NERC regions aren't strictly based on geography. Instead, I've taken the NERC codes from EIA-860 and assigned unknown plants (mostly those that retired before 2012 when modern NERCs were mostly defined, and those that were assigned a plant id in 2017) NERC labels using a k-nearest neighbors algorithm and lat/lon information.

See the `Assign NERC region labels` notebook for the creation of this file.

## Fraction of estimated gen/fuels in each NERC region

**NOTE** EIA changed the reporting requirements for wind and solar facilities in 2017. I'm now going to use the list of annual facilities from 2017 rather than from 2015, but still use the reported generation by those facilities in 2015.

In [76]:
%aimport Data.make_data
from Data.make_data import get_annual_plants

In [77]:
annual_ids_2015 = get_annual_plants(2015)

In [78]:
annual_ids_2017 = get_annual_plants(2017)

Combine the lists of annual plants in 2015 and 2017. This lets us catch facilities that have gone from monthly to annual since 2015, but it also includes plants that were annual in 2015 an may have retired.

There is the possibility of an error in allocation to NERC regions for 2016 state-level generation when using the facilities that changed to annual in 2017. But since the state-level generation is so much smaller in 2016 I don't think this is much of an issue.

In [79]:
annual_ids = set(annual_ids_2015.tolist() + annual_ids_2017.tolist())

In [80]:
len(annual_ids)

7146

### Add NERC region labels

In [81]:
%aimport Analysis.state2nerc
from Analysis.state2nerc import fraction_state2nerc, add_region

In [82]:
cwd = os.getcwd()
path = join(data_path, 'Facility labels',
            'Facility locations_knn.csv')
location_labels = pd.read_csv(path)

In [83]:
nerc_state_path = join(data_path, 'Derived data',
                  'NERC_states.json')

with open(nerc_state_path, 'r') as f:
    nerc_states = json.load(f)

Added the filter that year must be 2015 - was getting all 2015 annual plants, but for all years!

In [84]:
eia_2015_annual = eia_fac.loc[(eia_fac['plant id'].isin(annual_ids)) & 
                              (eia_fac['year'] == 2015)].copy()

# Group to state-level fuel categories
eia_2015_annual = group_fuel_cats(eia_2015_annual, state_fuel_cat)

In [85]:
eia_2015_annual_nerc = add_facility_location(eia_2015_annual, location_labels, 
                                        labels=['state', 'nerc'])

This is 2015 data on annual reporting facilities (from both 2015 and 2017)

In [86]:
eia_2015_annual_nerc.tail()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),state,nerc
92433,WWW,2015,8,58707,211987.0,13966.0,211987.0,33.238611,-80.450278,3,0.0,0.0,19884380.6,19884380.6,SC,RFC
92434,WWW,2015,9,58707,214505.0,13769.0,214505.0,33.238611,-80.450278,3,0.0,0.0,20120569.0,20120569.0,SC,RFC
92435,WWW,2015,10,58707,162426.0,8554.0,162426.0,33.238611,-80.450278,4,0.0,0.0,15235558.8,15235558.8,SC,RFC
92436,WWW,2015,11,58707,245845.0,13352.0,245845.0,33.238611,-80.450278,4,0.0,0.0,23060261.0,23060261.0,SC,RFC
92437,WWW,2015,12,58707,249553.0,14099.0,249553.0,33.238611,-80.450278,4,0.0,0.0,23408071.4,23408071.4,SC,RFC


In [87]:
# Get a list of all state abbreviations

all_states = []
for value in nerc_states.values():
    all_states.extend(value)
all_states = set(all_states)

In [88]:
df_list = []

for state in all_states:
    try:
        df_list.append(fraction_state2nerc(eia_2015_annual_nerc,
                       state, region_col='nerc', fuel_col='type'))
    except:
        print(state)
        pass

In [89]:
nerc_fraction = pd.concat(df_list)
nerc_fraction.set_index(['state', 'nerc', 'type'], inplace=True)
nerc_fraction.sort_index(inplace=True)

In [90]:
nerc_fraction.loc[idx['TX', :, 'WND'], :]

% generation  % total fuel  % elec fuel
state nerc type                                         
TX    SPP  WND       0.125231      0.125231     0.125231
      TRE  WND       0.874769      0.874769     0.874769
      WECC WND       0.000000      0.000000     0.000000

With the values below I can allocate extra state-level generation and fuel use to each of the NERC regions!

In [91]:
nerc_fraction.loc['TX']

% generation  % total fuel  % elec fuel
nerc type                                         
SERC HYC       0.260935      0.260935     0.260935
     NG        0.124994      0.153379     0.092145
     OOG       0.280953      0.678359     0.159521
     OTH      -0.006302      0.025416     0.006614
     WAS       0.032216      0.022106     0.031110
     WWW      -0.492566      0.190640     0.257302
SPP  NG        0.160722      0.174954     0.185692
     OOG       0.603794      0.200676     0.759801
     PEL       0.478368      0.745665     0.290000
     WAS       0.140836      0.382821     0.131444
     WND       0.125231      0.125231     0.125231
     WWW       0.609573      0.282686     0.398931
TRE  HYC       0.739065      0.739065     0.739065
     NG        0.652469      0.615575     0.650881
     OOG       0.115253      0.120965     0.080679
     OTH       1.006302      0.974584     0.993386
     PC        1.000000      1.000000     1.000000
     PEL       0.487887      0.245383     0.677393
     SUN       0.932089      0.932089     0.932089
     WAS       0.826947      0.595073     0.837446
     WND       0.874769      0.874769     0.874769
     WWW       0.882992      0.526674     0.343767
WECC NG        0.061815      0.056091     0.071281
     PEL       0.033746      0.008952     0.032607
     SUN       0.067911      0.067911     0.067911
     WND       0.000000      0.000000     0.000000

In [92]:
nerc_fraction.tail()

% generation  % total fuel  % elec fuel
state nerc type                                         
WY    WECC NG             1.0           1.0          1.0
           OOG            1.0           1.0          1.0
           OTH            1.0           1.0          1.0
           PEL            1.0           1.0          1.0
           WND            1.0           1.0          1.0

Making sure that no values are greater than 1 (within tolerance)

In [93]:
(nerc_fraction.groupby(['state', 'type']).sum() > 1.0001).any()

% generation    False
% total fuel    False
% elec fuel     False
dtype: bool

In [94]:
(nerc_fraction.groupby(['state', 'type']).sum()
 .loc[(nerc_fraction.groupby(['state', 'type']).sum() > 1).any(axis=1)])

,,% generation,% total fuel,% elec fuel
state,type,,,
AR,NG,1.0,1.0,1.0
TX,PEL,1.0,1.0,1.0


## Allocate extra gen from the state-level to regions

I still need to generate state-level total generation and fuel use!

In [95]:
idx = pd.IndexSlice

In [96]:
# a dictionary to match column names
nerc_frac_match = {'% generation': 'generation (mwh)',
                   '% total fuel': 'total fuel (mmbtu)',
                   '% elec fuel': 'elec fuel (mmbtu)'}

### Load state-level total gen/fuel consumption

In [98]:
path = join(data_path, 'Derived data',
            'EIA state-level gen fuel CO2 {}.csv'.format(file_date))

state_total = pd.read_csv(path, parse_dates=['datetime'])

In [99]:
rename_cols(state_total)
state_total['state'] = state_total['geography'].str[-2:]

In [100]:
state_total.head()

,type,year,month,geography,end,f,last_updated,sector,series_id,start,units,generation (mwh),total fuel (mmbtu),elec fuel (mmbtu),all fuel co2 (kg),elec fuel co2 (kg),datetime,quarter,state
0,AOR,2001,1,USA-AK,201712,M,2018-02-28T02:03:13-05:00,99,ELEC.GEN.AOR-AK-99.M,200101,megawatthours,87.00,NaN,NaN,NaN,NaN,2001-01-01,1,AK
1,AOR,2001,1,USA-AL,201712,M,2018-02-28T02:03:13-05:00,99,ELEC.GEN.AOR-AL-99.M,200101,megawatthours,401167.59,NaN,NaN,NaN,NaN,2001-01-01,1,AL
2,AOR,2001,1,USA-AR,201712,M,2018-02-28T02:03:13-05:00,99,ELEC.GEN.AOR-AR-99.M,200101,megawatthours,136530.37,NaN,NaN,NaN,NaN,2001-01-01,1,AR
3,AOR,2001,1,USA-AZ,201712,M,2018-02-28T02:03:13-05:00,99,ELEC.GEN.AOR-AZ-99.M,200101,megawatthours,453.00,NaN,NaN,NaN,NaN,2001-01-01,1,AZ
4,AOR,2001,1,USA-CA,201712,M,2018-02-28T02:03:13-05:00,99,ELEC.GEN.AOR-CA-99.M,200101,megawatthours,1717398.41,NaN,NaN,NaN,NaN,2001-01-01,1,CA


In [101]:
state_total.dtypes

type                          object
year                           int64
month                          int64
geography                     object
end                            int64
f                             object
last_updated                  object
sector                         int64
series_id                     object
start                          int64
units                         object
generation (mwh)             float64
total fuel (mmbtu)           float64
elec fuel (mmbtu)            float64
all fuel co2 (kg)            float64
elec fuel co2 (kg)           float64
datetime              datetime64[ns]
quarter                        int64
state                         object
dtype: object

Simplify the dataframe

In [102]:
cols = list(nerc_frac_match.values())
state_total = state_total.groupby(['state', 'year', 'month', 'type'])[cols].sum()
state_total.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AK    2001 1     AOR               87.0                 0.0                0.0
                 COW            46903.0           1120000.0           872000.0
                 HYC           104549.0                 0.0                0.0
                 NG            367521.0           4091000.0          3989000.0
                 PEL            71085.0            767000.0           763000.0

In [103]:
# list of NERC regions
nercs = nerc_fraction.index.get_level_values('nerc').unique()

### Group the facility data to state fuel categories and add state labels

In [104]:
cols = list(nerc_frac_match.values())
eia_fac_type = group_fuel_cats(eia_fac, state_fuel_cat)
eia_fac_type = add_facility_location(eia_fac_type, location_labels, ['state'])
eia_fac_type = eia_fac_type.groupby(['state', 'year', 'month', 'type'])[cols].sum()

In [105]:
eia_fac_type.loc[idx['OK', 2017, :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
OK    2017 1     WND           982955.0           9074640.0          9074640.0
           2     WND          1053595.0           9726789.0          9726789.0
           3     WND          1237665.0          11426122.0         11426122.0
           4     WND          1164423.0          10749954.0         10749954.0
           5     WND           960103.0           8863670.0          8863670.0
           6     WND           987307.0           9114817.0          9114817.0
           7     WND           741231.0           6843046.0          6843046.0
           8     WND           496488.0           4583577.0          4583577.0
           9     WND           942175.0           8698158.0          8698158.0
           10    WND          1316521.0          12154122.0         12154122.0
           11    WND          1049365.0           9687738.0          9687738.0
           12    WND          1105605.0          10206947.0         10206947.0

In [106]:
eia_fac_type.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AK    2001 1     COW          26493.302           576574.00          520312.00
                 HYC         103940.302          1074015.14         1074015.14
                 NG          314206.726          3372210.00         3270620.00
                 PEL          65373.158           707882.00          704150.00
                 WND             86.518              893.99             893.99

### Calculate the extra gen/fuel consumption at the state levels
**Only worrying about extra generation from 2016 forward.**

In [107]:
state_extra = (state_total.loc[idx[:, 2016:, :, :], :]
               - eia_fac_type.loc[idx[:, 2016:, :, :], :])
state_extra.dropna(how='all', inplace=True)
state_extra = state_extra.reorder_levels(['year', 'state', 'month', 'type'])
state_extra.sort_index(inplace=True)
state_extra.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AK    1     COW              0.000                 0.0                1.0
                 HYC              0.004          -1451631.0         -1451631.0
                 NG           -7127.760            -93858.0           -93863.0
                 PEL             -0.001                -4.0                0.0
                 WAS             -0.001            -27910.0           -27910.0

The huge jump in Texas state-level wind generation is why I need to use more recent lists of annual reporting facilities rather than the list from 2015.

In [108]:
state_extra.loc[idx[:, 'TX', :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 TX    1     WND        -112214.996         -42126843.0        -42126843.0
           2     WND        -125883.003         -48434096.0        -48434096.0
           3     WND        -151810.997         -53426416.0        -53426416.0
           4     WND        -129348.002         -44924684.0        -44924684.0
           5     WND        -135324.999         -49005417.0        -49005417.0
           6     WND         -95285.004         -35791076.0        -35791076.0
           7     WND        -118644.004         -53486104.0        -53486104.0
           8     WND         -70829.002         -34831264.0        -34831264.0
           9     WND         -83498.000         -36917999.0        -36917999.0
           10    WND        -118149.998         -51416318.0        -51416318.0
           11    WND        -105434.001         -42667856.0        -42667856.0
           12    WND        -117457.004         -50687459.0        -50687459.0
2017 TX    1     WND        2834795.960         -28034705.0        -28034705.0
           2     WND        2773018.990         -27963387.0        -27963387.0
           3     WND        3298962.210         -34012618.0        -34012618.0
           4     WND        3249599.850         -33272161.0        -33272161.0
           5     WND        2890690.220         -30894588.0        -30894588.0
           6     WND        2373841.520         -23446234.0        -23446234.0
           7     WND        2033387.510         -21834471.0        -21834471.0
           8     WND        1828367.270         -17547499.0        -17547499.0
           9     WND        2257337.910         -23633911.0        -23633911.0
           10    WND        2880387.030         -30043631.0        -30043631.0
           11    WND        2610271.050         -29996768.0        -29996768.0
           12    WND        2503670.430         -27592945.0        -27592945.0

Why is generation from facilities greater than state-level estimates in some cases?

In [109]:
state_extra.loc[(state_extra < -100).any(axis=1)]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AK    1     HYC       4.000000e-03          -1451631.0         -1451631.0
                 NG       -7.127760e+03            -93858.0           -93863.0
                 WAS      -1.000000e-03            -27910.0           -27910.0
                 WND      -1.114910e+02            -40223.0           -40223.0
           2     HYC       1.000000e-03          -1197011.0         -1197011.0
                 NG       -6.190797e+03            -81521.0           -81526.0
                 WAS      -4.000000e-03            -20262.0           -20262.0
                 WND      -2.100260e+02            -75774.0           -75774.0
           3     HYC       3.000000e-03          -1529981.0         -1529981.0
                 NG       -6.518067e+03            -85829.0           -85835.0
                 WAS       5.000000e-03            -34523.0           -34523.0
                 WND      -2.450940e+02            -88426.0           -88426.0
           4     HYC       4.000000e-03           -960217.0          -960217.0
                 NG       -6.472865e+03            -85240.0           -85242.0
                 WAS      -3.000000e-03            -32713.0           -32713.0
                 WND      -4.746720e+02           -171253.0          -171253.0
           5     HYC      -2.000000e-03           -636014.0          -636014.0
                 NG       -6.470901e+03            -85212.0           -85214.0
                 WAS       0.000000e+00            -35453.0           -35453.0
                 WND      -3.483280e+02           -125672.0          -125672.0
           6     HYC      -2.000000e-03           -737345.0          -737345.0
                 NG       -5.928666e+03            -78068.0           -78074.0
                 WAS       4.547474e-13            -35304.0           -35304.0
                 WND      -4.767630e+02           -172009.0          -172009.0
           7     HYC      -4.000000e-03           -956769.0          -956769.0
                 NG       -6.321044e+03            -83240.0           -83240.0
                 WAS      -4.547474e-13            -34835.0           -34835.0
                 WND      -4.255800e+02           -153544.0          -153544.0
           8     HYC      -3.000000e-03          -1388457.0         -1388457.0
                 NG       -5.398927e+03            -71094.0           -71091.0
...                                 ...                 ...                ...
2017 WY    1     OOG       3.499000e+00           -688106.0          -235324.0
                 OTH      -4.000000e-03           -242464.0           -24246.0
           2     OOG       5.977940e+02           -597907.0          -214575.0
                 OTH      -1.000000e-03           -242097.0           -23080.0
           3     OOG       6.245260e+02           -609638.0          -217438.0
                 OTH       1.000000e-03           -263803.0           -22660.0
           4     HYC       1.062640e+05              -832.0             -832.0
                 OOG       6.848850e+02           -611558.0          -216818.0
                 OTH       2.000000e-03           -213258.0           -19563.0
           5     OOG       1.275420e+02           -522349.0          -171698.0
                 OTH       0.000000e+00           -156847.0                0.0
           6     HYC       1.143932e+05            -16625.0           -16625.0
                 OOG       8.111630e+02           -467382.0          -148542.0
                 OTH       0.000000e+00           -257499.0                0.0
           7     HYC       8.330886e+04            -17605.0           -17605.0
                 OOG       7.435590e+02           -471614.0          -147534.0
                 OTH       0.000000e+00           -260675.0                0.0
           8     HYC       6.162763e+04            -17256.0           -17

Sort the index of each dataframe to make sure they can be easily combined.

In [110]:
nerc_fraction.sort_index(inplace=True)
nerc_fraction.loc[idx['TX', 'WECC', :], :].tail()

% generation  % total fuel  % elec fuel
state nerc type                                         
TX    WECC NG        0.061815      0.056091     0.071281
           PEL       0.033746      0.008952     0.032607
           SUN       0.067911      0.067911     0.067911
           WND       0.000000      0.000000     0.000000

In [111]:
state_extra.sort_index(inplace=True)
state_extra.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AK    1     COW              0.000                 0.0                1.0
                 HYC              0.004          -1451631.0         -1451631.0
                 NG           -7127.760            -93858.0           -93863.0
                 PEL             -0.001                -4.0                0.0
                 WAS             -0.001            -27910.0           -27910.0

Create a copy of the `nerc_fraction` dataframe with repeated values for every month of the year, so that they MultiIndex matches the `state_extra` MultiIndex

In [112]:
df_list = []
for month in range(1, 13):
    df = nerc_fraction.copy()
    df['month'] = month
    df.set_index('month', append=True, inplace=True)
    df_list.append(df)

nerc_frac_monthly = pd.concat(df_list, axis=0)
nerc_frac_monthly.sort_index(inplace=True)
nerc_frac_monthly = (nerc_frac_monthly
                     .reorder_levels(['nerc', 'state', 'month', 'type']))

In [113]:
nercs

Index(['SERC', 'SPP', 'WECC', 'MRO', 'NPCC', 'RFC', 'FRCC', 'TRE'], dtype='object', name='nerc')

Cycle through each year (2016 and 2017 in this case) and each NERC, multiplying the state-level extra generation, total fuel consumption, and fuel consumption for electricity by the share that should be allocated to each NERC.

In [114]:
df_list_outer = []
for year in [2016, 2017]:
    df_list_inner = []
    for nerc in nercs:
        df = pd.concat([(nerc_frac_monthly
                         .loc[nerc]['% generation']
                         * state_extra
                         .loc[year]['generation (mwh)']).dropna(),
                        (nerc_frac_monthly.
                         loc[nerc]['% total fuel']
                         * state_extra
                         .loc[year]['total fuel (mmbtu)']).dropna(),
                        (nerc_frac_monthly
                         .loc[nerc]['% elec fuel']
                         * state_extra
                         .loc[year]['elec fuel (mmbtu)']).dropna()],
                        axis=1)
        df.columns = nerc_frac_match.values()
        df['nerc'] = nerc
        df['year'] = year
        df = df.groupby(['year', 'nerc', 'month', 'type']).sum()
        df_list_inner.append(df)

    df_list_outer.append(pd.concat(df_list_inner))
final = pd.concat(df_list_outer)
final.sort_index(inplace=True)

In [115]:
nerc_frac_monthly.sort_index(inplace=True)

Although the dataframe is called `final`, it's really just the final allocated extra state-level generation/fuel consumption

In [116]:
final.loc[idx[2017, 'SPP', :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year nerc month type                                                         
2017 SPP  1     WND       2.225204e+06       -1.972301e+07      -1.972301e+07
          2     WND       2.262461e+06       -2.049096e+07      -2.049096e+07
          3     WND       2.650092e+06       -2.563555e+07      -2.563555e+07
          4     WND       2.571236e+06       -2.534980e+07      -2.534980e+07
          5     WND       2.235343e+06       -2.173511e+07      -2.173511e+07
          6     WND       1.870060e+06       -2.035920e+07      -2.035920e+07
          7     WND       1.531402e+06       -1.634142e+07      -1.634142e+07
          8     WND       1.277675e+06       -1.202808e+07      -1.202808e+07
          9     WND       1.819973e+06       -2.033756e+07      -2.033756e+07
          10    WND       2.391769e+06       -2.616970e+07      -2.616970e+07
          11    WND       2.186464e+06       -2.246211e+07      -2.246211e+07
          12    WND       2.176394e+06       -2.351710e+07      -2.351710e+07

In [118]:
path = join(data_path, 'Derived data',
            'NERC extra gen fuels {}.csv'.format(file_date))
final.to_csv(path)